In [1]:
#inspiration from here https://www.learndatasci.com/tutorials/reinforcement-q-learning-scratch-python-openai-gym/
!pip install cmake 'gym[atari]' scipy

    100% |████████████████████████████████| 36.4MB 384kB/s 
    100% |████████████████████████████████| 1.5MB 2.9MB/s 
    100% |████████████████████████████████| 1.0MB 3.5MB/s 
    100% |████████████████████████████████| 3.3MB 4.3MB/s 
    100% |████████████████████████████████| 1.2MB 6.1MB/s 
    100% |████████████████████████████████| 829kB 6.1MB/s 
  Running setup.py bdist_wheel for gym ... - \ | / - done
  Stored in directory: /Users/chloeloughridge/Library/Caches/pip/wheels/57/b0/13/4153e1acab826fbe612c95b1336a63a3fa6416902a8d74a1b7
  Running setup.py bdist_wheel for PyOpenGL ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | done
  Stored in directory: /Users/chloeloughridge/Library/Caches/pip/wheels/6c/00/7f/1dd736f380848720ad79a1a1de5272e0d3f79c15a42968fb58
  Running setup.py bdist_wheel for future ... - \ | / done
  Stored in directory: /Users/chloeloughridge/Library/Caches/pip/wheels/0c/61/d2/d6b7317325828fbb39ee

In [1]:
import gym

In [2]:
env = gym.make("Taxi-v2").env
env.render()

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [3]:
env.P[328]

{0: [(1.0, 428, -1, False)],
 1: [(1.0, 228, -1, False)],
 2: [(1.0, 348, -1, False)],
 3: [(1.0, 328, -1, False)],
 4: [(1.0, 328, -10, False)],
 5: [(1.0, 328, -10, False)]}

In [6]:
# navigating the environment randomly first
env.s = 328
epochs = 0
penalties, reward = 0, 0
frames = []
done = False

while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)
    
    if reward == -10:
        penalties += 1
    
    frames.append({
            'frame': env.render(mode='ansi'),
            'state': state,
            'action':action,
            'reward': reward
        })
    epochs += 1
    
print("timesteps taken: {}".format(epochs))
print("penalties incurred: {}".format(penalties))
print("rewards: {}".format(reward))

timesteps taken: 864
penalties incurred: 285
rewards: 20


In [12]:
#animation!
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print("Timestep: {i + 1}")
        print("State: {frame['state']}")
        print("Action: {frame['action']}")
        print("Reward: {frame['reward']}")
        sleep(.1)
        
print_frames(frames)

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)

Timestep: {i + 1}
State: {frame['state']}
Action: {frame['action']}
Reward: {frame['reward']}


KeyboardInterrupt: 

In [13]:
# Now building a Q-learning agent
import numpy as np
#initialize Q-table
q_table = np.zeros([env.observation_space.n, env.action_space.n])

In [17]:
import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

# continuously updating Q-table until Q-values converge on best policy

for i in range(1, 100001):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0,1) < epsilon:
            action = env.action_space.sample() #explore
        else:
            action = np.argmax(q_table[state]) #exploit
        
        next_state, reward, done, info = env.step(action)
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1-alpha)*old_value + alpha*(reward+gamma*next_max)
        q_table[state, action] = new_value
        
        if reward == -10:
            penalties += 1
            
        state = next_state
        epochs += 1
        
        if i % 100 == 0:
            clear_output(wait=True)
            print("Episode: {}".format(i))
            
print("training finished")

Episode: 100000
training finished


In [25]:
# Evaluating the agent's performance
total_epochs, total_penalties, total_rewards = 0, 0, 0
episodes = 100
episode_frames = []

for i in range(episodes):
    state = env.reset()
    
    frames = []
    
    epochs, penalties, reward = 0,0,0
    
    done = False

    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)
    
        if reward == -10:
            penalties += 1
    
        frames.append({
            'frame': env.render(mode='ansi'),
            'state': state,
            'action':action,
            'reward': reward
        })
        
        epochs += 1
    
    episode_frames.append(frames)
        
    total_penalties += penalties
    total_rewards += reward
    total_epochs += epochs
    
print("timesteps taken: {}".format(total_epochs))
print("penalties incurred: {}".format(total_penalties))
print("rewards: {}".format(total_rewards))

timesteps taken: 1239
penalties incurred: 0
rewards: 2000


In [29]:
# animating the Q-learning agent --> animates episode by episode
for i in range(episodes):
    clear_output(wait=True)
    print("Episode: {}".format(i))
    sleep(1)
    print_frames(episode_frames[i])

Episode: 13


KeyboardInterrupt: 